In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

options = Options()
# options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)

print("Cargando página...")
driver.get("https://www.promofarma.com/cosmetica/facial?page=1")

print("Esperando que se acomoden los productos...")
time.sleep(8)

try:
    print("Buscando productos...")
    productos = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3.TextContent--small"))
    )
    print(f"{len(productos)} productos encontrados.")

    driver.execute_script("arguments[0].scrollIntoView(true);", productos[0])
    time.sleep(1)
    driver.execute_script("arguments[0].click();", productos[0])

    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "h1.TextDisplay--medium"))
    )
    time.sleep(2)

    nombre = driver.find_element(By.CSS_SELECTOR, "h1.TextDisplay--medium").text

    try:
        precio = driver.find_element(By.CSS_SELECTOR, "span[data-price]").get_attribute("data-price").strip()
    except:
        precio = ""

    imagen_el = driver.find_element(By.CSS_SELECTOR, 'img[data-qa-ta="image"]')
    imagen_url = imagen_el.get_attribute("src")

    url_producto = driver.current_url

    # Hacer scroll y clic en "Composición"
    composicion_btn = driver.find_element(By.XPATH, "//h2[contains(text(), 'Composición')]")
    driver.execute_script("arguments[0].scrollIntoView(true);", composicion_btn)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", composicion_btn)

    # Esperar que el contenedor padre del h2 "Composición" se expanda y el <p> dentro esté visible
    ingredientes = ""
    try:
        contenedor_composicion = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//h2[contains(text(),'Composición')]/parent::*"))
        )
        # Ahora buscar el <p> dentro de ese contenedor
        p_ingredientes = contenedor_composicion.find_element(By.TAG_NAME, "p")
        ingredientes = p_ingredientes.text.strip()
    except:
        ingredientes = "No encontrado"

    print("Nombre:", nombre)
    print("Precio:", precio)
    print("Imagen:", imagen_url)
    print("URL producto:", url_producto)
    print("Ingredientes:", ingredientes)

except Exception as e:
    print("Error durante el scraping:", e)

driver.quit()

Cargando página...
Esperando que se acomoden los productos...
Buscando productos...
64 productos encontrados.
Nombre: Avène Hydrance UV Rica Crema Hidratante SPF30 40ml
Precio: 23,89€
Imagen: https://statics.promofarma.com/static/promofarma/prod/product_images/mr/ZYHAL_es_ES_0.jpg
URL producto: https://www.promofarma.com/es/avene-hydrance-uv-rica-crema-hidratante-spf30-40ml/p-8345
Ingredientes: AVENE THERMAL SPRING WATER (AVENE AQUA). C12-15 ALKYL BENZOATE. DICAPRYLYL CARBONATE. DIISOPROPYL ADIPATE. GLYCERIN. METHYLENE BIS-BENZOTRIAZOLYL TETRAMETHYLBUTYLPHENOL [NANO]. WATER (AQUA). BIS-ETHYLHEXYLOXYPHENOL METHOXYPHENYL TRIAZINE. DIETHYLHEXYL BUTAMIDO TRIAZONE. POLYMETHYL METHACRYLATE. BUTYL METHOXYDIBENZOYLMETHANE. BUTYROSPERMUM PARKII (SHEA) BUTTER (BUTYROSPERMUM PARKII BUTTER). METHYL GLUCETH-20. ISOCETYL STEAROYL STEARATE. TRIETHYLHEXANOIN. HYDROGENATED STARCH HYDROLYSATE. C10-18 TRIGLYCERIDES. GLYCERYL STEARATE. PEG-100 STEARATE. POTASSIUM CETYL PHOSPHATE. BENZOIC ACID. CAPRYLIC/CA

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

options = Options()
# options.add_argument('--headless')  # Descomenta si quieres sin ventana
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)

print("Cargando página...")
driver.get("https://www.promofarma.com/cosmetica/facial?page=1")

print("Esperando que se acomoden los productos...")
time.sleep(8)

try:
    productos = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3.TextContent--small"))
    )
    print(f"{len(productos)} productos encontrados.")

    productos_info = []

    for i, producto in enumerate(productos):
        print(f"\nProcesando producto {i+1}/{len(productos)}")
        try:
            # Abrir el producto en una pestaña nueva para no perder la lista
            url_producto = producto.find_element(By.XPATH, "..").get_attribute("href")
            driver.execute_script("window.open(arguments[0]);", url_producto)
            driver.switch_to.window(driver.window_handles[1])

            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h1.TextDisplay--medium"))
            )
            time.sleep(2)

            nombre = driver.find_element(By.CSS_SELECTOR, "h1.TextDisplay--medium").text

            try:
                precio = driver.find_element(By.CSS_SELECTOR, "span[data-price]").get_attribute("data-price").strip()
            except:
                precio = ""

            imagen_el = driver.find_element(By.CSS_SELECTOR, 'img[data-qa-ta="image"]')
            imagen_url = imagen_el.get_attribute("src")

            # Clic en "Composición"
            composicion_btn = driver.find_element(By.XPATH, "//h2[contains(text(), 'Composición')]")
            driver.execute_script("arguments[0].scrollIntoView(true);", composicion_btn)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", composicion_btn)

            ingredientes = ""
            try:
                contenedor_composicion = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, "//h2[contains(text(),'Composición')]/parent::*"))
                )
                p_ingredientes = contenedor_composicion.find_element(By.TAG_NAME, "p")
                ingredientes = p_ingredientes.text.strip()
            except:
                ingredientes = "No encontrado"

            productos_info.append({
                "nombre": nombre,
                "precio": precio,
                "imagen_url": imagen_url,
                "url_producto": url_producto,
                "ingredientes": ingredientes
            })

            driver.close()
            driver.switch_to.window(driver.window_handles[0])

        except Exception as e:
            print(f"Error con producto {i+1}: {e}")
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            continue

    print("\n--- Resultados ---")
    for info in productos_info:
        print(f"Nombre: {info['nombre']}")
        print(f"Precio: {info['precio']}")
        print(f"Imagen: {info['imagen_url']}")
        print(f"URL: {info['url_producto']}")
        print(f"Ingredientes: {info['ingredientes']}")
        print("-" * 40)

except Exception as e:
    print("Error general:", e)

driver.quit()

Cargando página...
Esperando que se acomoden los productos...
64 productos encontrados.

Procesando producto 1/64

Procesando producto 2/64

Procesando producto 3/64

Procesando producto 4/64

Procesando producto 5/64

Procesando producto 6/64

Procesando producto 7/64

Procesando producto 8/64

Procesando producto 9/64

Procesando producto 10/64

Procesando producto 11/64

Procesando producto 12/64

Procesando producto 13/64

Procesando producto 14/64

Procesando producto 15/64

Procesando producto 16/64

Procesando producto 17/64

Procesando producto 18/64

Procesando producto 19/64

Procesando producto 20/64

Procesando producto 21/64

Procesando producto 22/64

Procesando producto 23/64

Procesando producto 24/64

Procesando producto 25/64

Procesando producto 26/64

Procesando producto 27/64

Procesando producto 28/64

Procesando producto 29/64

Procesando producto 30/64

Procesando producto 31/64

Procesando producto 32/64

Procesando producto 33/64

Procesando producto 34/64

Pr

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

options = Options()
# options.add_argument('--headless')  # Si quieres ocultar la ventana, descomenta esto
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_experimental_option("detach", True)  # <-- Evita que se cierre la ventana al terminar

driver = webdriver.Chrome(options=options)

print("Cargando página...")
driver.get("https://www.promofarma.com/cosmetica/facial?page=1")

print("Esperando que se acomoden los productos...")
time.sleep(8)

try:
    productos = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3.TextContent--small"))
    )
    print(f"{len(productos)} productos encontrados.")

    productos_info = []

    for i, producto in enumerate(productos):
        print(f"\nProcesando producto {i+1}/{len(productos)}")
        try:
            url_producto = producto.find_element(By.XPATH, "..").get_attribute("href")
            driver.execute_script("window.open(arguments[0]);", url_producto)
            driver.switch_to.window(driver.window_handles[1])

            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h1.TextDisplay--medium"))
            )
            time.sleep(2)

            nombre = driver.find_element(By.CSS_SELECTOR, "h1.TextDisplay--medium").text

            try:
                precio = driver.find_element(By.CSS_SELECTOR, "span[data-price]").get_attribute("data-price").strip()
            except:
                precio = ""

            imagen_el = driver.find_element(By.CSS_SELECTOR, 'img[data-qa-ta="image"]')
            imagen_url = imagen_el.get_attribute("src")

            composicion_btn = driver.find_element(By.XPATH, "//h2[contains(text(), 'Composición')]")
            driver.execute_script("arguments[0].scrollIntoView(true);", composicion_btn)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", composicion_btn)

            ingredientes = ""
            try:
                contenedor_composicion = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, "//h2[contains(text(),'Composición')]/parent::*"))
                )
                p_ingredientes = contenedor_composicion.find_element(By.TAG_NAME, "p")
                ingredientes = p_ingredientes.text.strip()
            except:
                ingredientes = "No encontrado"

            productos_info.append({
                "nombre": nombre,
                "precio": precio,
                "imagen_url": imagen_url,
                "url_producto": url_producto,
                "ingredientes": ingredientes
            })

            driver.close()
            driver.switch_to.window(driver.window_handles[0])

        except Exception as e:
            print(f"Error con producto {i+1}: {e}")
            # Asegurarse que siempre volvemos a la ventana principal
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            continue

    print("\n--- Resultados ---")
    for info in productos_info:
        print(f"Nombre: {info['nombre']}")
        print(f"Precio: {info['precio']}")
        print(f"Imagen: {info['imagen_url']}")
        print(f"URL: {info['url_producto']}")
        print(f"Ingredientes: {info['ingredientes']}")
        print("-" * 40)

except Exception as e:
    print("Error general:", e)

# No cerramos driver.quit() para que no se cierre la ventana automáticamente.

Cargando página...
Esperando que se acomoden los productos...
64 productos encontrados.

Procesando producto 1/64

Procesando producto 2/64

Procesando producto 3/64

Procesando producto 4/64

Procesando producto 5/64

Procesando producto 6/64

Procesando producto 7/64

Procesando producto 8/64

Procesando producto 9/64

Procesando producto 10/64

Procesando producto 11/64

Procesando producto 12/64

Procesando producto 13/64

Procesando producto 14/64

Procesando producto 15/64

Procesando producto 16/64

Procesando producto 17/64

Procesando producto 18/64

Procesando producto 19/64

Procesando producto 20/64

Procesando producto 21/64

Procesando producto 22/64

Procesando producto 23/64

Procesando producto 24/64

Procesando producto 25/64

Procesando producto 26/64

Procesando producto 27/64

Procesando producto 28/64

Procesando producto 29/64

Procesando producto 30/64

Procesando producto 31/64

Procesando producto 32/64

Procesando producto 33/64

Procesando producto 34/64

Pr

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

options = Options()
# options.add_argument('--headless')  # Descomenta si quieres sin ventana
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_experimental_option("detach", True)  # Mantiene la ventana abierta

driver = webdriver.Chrome(options=options)

productos_info = []

for pagina in range(1, 11):
    print(f"\nCargando página {pagina}...")
    driver.get(f"https://www.promofarma.com/cosmetica/facial?page={pagina}")

    print("Esperando que se acomoden los productos...")
    time.sleep(8)

    try:
        productos = WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3.TextContent--small"))
        )
        print(f"{len(productos)} productos encontrados en la página {pagina}.")

        for i, producto in enumerate(productos):
            print(f"\nProcesando producto {i+1}/{len(productos)} de página {pagina}")
            try:
                url_producto = producto.find_element(By.XPATH, "..").get_attribute("href")
                driver.execute_script("window.open(arguments[0]);", url_producto)
                driver.switch_to.window(driver.window_handles[1])

                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h1.TextDisplay--medium"))
                )
                time.sleep(2)

                nombre = driver.find_element(By.CSS_SELECTOR, "h1.TextDisplay--medium").text

                try:
                    precio = driver.find_element(By.CSS_SELECTOR, "span[data-price]").get_attribute("data-price").strip()
                except:
                    precio = ""

                imagen_el = driver.find_element(By.CSS_SELECTOR, 'img[data-qa-ta="image"]')
                imagen_url = imagen_el.get_attribute("src")

                composicion_btn = driver.find_element(By.XPATH, "//h2[contains(text(), 'Composición')]")
                driver.execute_script("arguments[0].scrollIntoView(true);", composicion_btn)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", composicion_btn)

                ingredientes = ""
                try:
                    contenedor_composicion = WebDriverWait(driver, 10).until(
                        EC.visibility_of_element_located((By.XPATH, "//h2[contains(text(),'Composición')]/parent::*"))
                    )
                    p_ingredientes = contenedor_composicion.find_element(By.TAG_NAME, "p")
                    ingredientes = p_ingredientes.text.strip()
                except:
                    ingredientes = "No encontrado"

                productos_info.append({
                    "nombre": nombre,
                    "precio": precio,
                    "ingredientes": ingredientes,
                    "url_producto": url_producto,
                    "imagen_url": imagen_url
                })

                driver.close()
                driver.switch_to.window(driver.window_handles[0])

            except Exception as e:
                print(f"Error con producto {i+1} en página {pagina}: {e}")
                if len(driver.window_handles) > 1:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                continue

    except Exception as e:
        print(f"Error general en página {pagina}: {e}")
        continue

# Guardar CSV
csv_file = "promofarma_products.csv"
print(f"\nGuardando datos en {csv_file}...")

with open(csv_file, mode='w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=["nombre", "precio", "ingredientes", "url_producto", "imagen_url"])
    writer.writeheader()
    for producto in productos_info:
        writer.writerow(producto)

print("Guardado completado.")

# No cerramos driver.quit() para mantener ventana abierta.


Cargando página 1...
Esperando que se acomoden los productos...
64 productos encontrados en la página 1.

Procesando producto 1/64 de página 1

Procesando producto 2/64 de página 1

Procesando producto 3/64 de página 1

Procesando producto 4/64 de página 1

Procesando producto 5/64 de página 1

Procesando producto 6/64 de página 1

Procesando producto 7/64 de página 1

Procesando producto 8/64 de página 1

Procesando producto 9/64 de página 1

Procesando producto 10/64 de página 1

Procesando producto 11/64 de página 1

Procesando producto 12/64 de página 1

Procesando producto 13/64 de página 1

Procesando producto 14/64 de página 1

Procesando producto 15/64 de página 1

Procesando producto 16/64 de página 1

Procesando producto 17/64 de página 1

Procesando producto 18/64 de página 1

Procesando producto 19/64 de página 1

Procesando producto 20/64 de página 1

Procesando producto 21/64 de página 1

Procesando producto 22/64 de página 1

Procesando producto 23/64 de página 1

Proce